In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import TensorDataset, DataLoader
from utils import load_dataset_df,  data_splitter, get_spiking_net, make_filename, smiles_to_feat
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from csnn_model import get_prediction_fn

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\snn_venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometri

#### Load DataFrame

In [2]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[0]
dirname = dt_file.removesuffix('.csv')

df, targets = load_dataset_df(filename=dt_file)


In [3]:
if dirname == 'tox21':
    # SR-ARE
    target_name = targets[7]
    # SR-MMP
elif dirname == 'sider':
    #Hepatobiliary disorders 1427 samples, 0.52 class ratio
    target_name = targets[0]
else:
    target_name = targets[0]
    
df = df[[target_name, 'smiles']].dropna()

#### Molecular Representation

In [4]:
representations = ["fp", "descriptor"]

repr_type = representations[0]

In [5]:
if repr_type == "fp":
    fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024], ['count_morgan', 1024], ['pubchem', 881]]
    mix = True
    fp_type, num_bits = fp_types[2]
    if mix and fp_type == 'RDKit':
        num_bits = 512
    data_config = {"fp_type": fp_type,
                "num_bits": num_bits,
                "radius": 2,
                "fp_type_2": fp_types[0][0],
                "num_bits_2": 1024 - num_bits,
                "mix": mix,
                "dim_2": False}
    dim_2 = data_config['dim_2']
    print(fp_type, '-', num_bits)
    if mix: print(data_config['fp_type_2'], '-', data_config['num_bits_2'])
    if dim_2: print("2D FP")

elif repr_type == "descriptor":
    mix = False
    desc_type = ["RDKit", "Mordred"]
    data_config = {"desc_type": desc_type[0],
                   "size": 0,
                }

data_config["repr_type"] = repr_type
print(repr_type)

RDKit - 512
morgan - 512
fp


In [6]:
dtype = torch.float
split = "scaffold" if dirname == "BBBP" else "random"
dataset = None


feat_tensor, target_tensor, feat_df = smiles_to_feat(df,repr_type=repr_type, data_config=data_config, target_name=target_name, dtype=dtype)
print(feat_tensor.shape)
dataset = TensorDataset(feat_tensor, target_tensor)

[09:18:16] Explicit valence for atom # 8 Al, 6, is greater than permitted
[09:18:17] Explicit valence for atom # 3 Al, 6, is greater than permitted
[09:18:17] Explicit valence for atom # 4 Al, 6, is greater than permitted
[09:18:18] Explicit valence for atom # 4 Al, 6, is greater than permitted
[09:18:19] Explicit valence for atom # 9 Al, 6, is greater than permitted
[09:18:19] Explicit valence for atom # 5 Al, 6, is greater than permitted
[09:18:20] Explicit valence for atom # 16 Al, 6, is greater than permitted


torch.Size([5825, 1024])


#### Loss Function

In [7]:
loss_types = ['ce_mem', 'rate_loss', 'count_loss', 'temporal_loss', 'bce_loss']
loss_type = loss_types[2]
print(loss_type)


count_loss


#### Test Loop

In [8]:
net_types = ["SNN", "DSNN", "CSNN", "RSNN"]
net_type = net_types[2]


In [9]:

slope = 10
#spike_grad = surrogate.fast_sigmoid(slope=slope)
spike_grad = None
beta = 0.95
bias = True
net_config = {
            "num_hidden": 1024,
            "num_hidden_l2": 256,
            "num_steps": 10,
            "spike_grad": spike_grad,
            "slope": None if not spike_grad else slope, #spike_grad.__closure__[0].cell_contents,
            "beta": beta,
            "encoding": 'rate' if loss_type != 'temporal_loss' else 'ttfs',
            "bias": bias,
            "out_num": 2,
            "num_hidden_layers": 2,
            "num_hidden_l3": 256,
            }
if net_type == "CSNN":
    net_config['num_conv'] = 1
    net_config["pool_size"] = 2
    net_config["conv_kernel"] = 3
    net_config["conv_stride"] = [1 for _ in range(net_config['num_conv'])]
    net_config["conv_groups"] = 1

if repr_type == "fp":
    net_config["input_size"] = 1024 if data_config['mix'] else num_bits
    net_config["2d"] = data_config['dim_2']

elif repr_type == "descriptor":
    net_config["input_size"] = feat_tensor.shape[1]
    net_config["2d"] = False
    #net_config["num_steps"] = 10

print(net_type)

CSNN


In [10]:
   
pop_coding = net_config['out_num'] > 2
lr=1e-4
iterations = 30
weight_decay = 0 # 1e-5
#weight_decay = 1e-4
optim_type = 'Adam'
#optim_type = 'SGD'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 16 #16, 8
scores = True
train_config = {"num_epochs": 100,
                "batch_size": batch_size,
                "device": device,
                "loss_type": loss_type,
                "loss_fn": None,
                'dtype': dtype,
                'num_steps': net_config['num_steps'],
                'val_net': None,
                'prediction_fn': get_prediction_fn(encoding=net_config['encoding'], pop_coding=pop_coding, scores=scores),
                }
drop_last = net_type == "CSNN"
pin_memory = device == "cuda"
save_csv = True
save_models = True
print(train_config["prediction_fn"])

cuda
<function prediction_spk_rate_scores at 0x0000027861A8E950>


In [11]:
calculate_scores = True

def calc_metrics(metrics_list, all_targets, all_preds):
    if calculate_scores:
        auc_roc = roc_auc_score(all_targets, all_preds)
        all_preds = np.array(all_preds) > 0.0
    else:
        all_preds = np.array(all_preds) > 0.0
        auc_roc = roc_auc_score(all_targets, all_preds)
    
    accuracy = accuracy_score(all_targets, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_targets, all_preds).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = f1_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds)
    print("acc:", accuracy, "auc:", auc_roc)
    metrics_list[0].append(accuracy)
    metrics_list[1].append(auc_roc)
    metrics_list[2].append(sensitivity)
    metrics_list[3].append(specificity)
    metrics_list[4].append(f1)
    metrics_list[5].append(precision)
    

In [12]:
import os
import copy

net_list = []
    
net, train_net, val_net, test_net = get_spiking_net(net_type, net_config)
filename = make_filename(dirname, target_name, net_type, data_config, lr, weight_decay, optim_type, net_config, train_config, net, model = True)

model_name = filename.removesuffix('.csv')

models_path = os.path.join("results", dirname, "models", "")
all_model_names = os.listdir(models_path)
print(models_path)
#print(all_model_names)
for iter in range(iterations):
    seed = int(iter + 1)
    string_id = f"seed-{seed}.pth"
    search_name = model_name + str(string_id) 
    search_name_no_folder = search_name.removeprefix(models_path)
    if search_name_no_folder in all_model_names:
        state_dict = torch.load(search_name, weights_only=True)
        net_list.append(copy.deepcopy(state_dict))
    else: print(search_name_no_folder)

1
results\tox21\models\


In [13]:
if len(net_list) < 30:
    net_list = []
    for iter in range(iterations):
        seed = int(iter + 1)
        string_id = f"-scores-seed-{seed}.pth"
        search_name = model_name + str(string_id) 
        search_name_no_folder = search_name.removeprefix(models_path)
        if search_name_no_folder in all_model_names:
            state_dict = torch.load(search_name, weights_only=True)
            net_list.append(copy.deepcopy(state_dict))

if len(net_list) < 30:
    net_list = []
    for iter in range(iterations):
        seed = int(iter + 1)
        string_id = f"seed-{seed}_auc.pth"
        search_name = model_name + str(string_id) 
        search_name_no_folder = search_name.removeprefix(models_path)
        if search_name_no_folder in all_model_names:
            state_dict = torch.load(search_name, weights_only=True)
            net_list.append(copy.deepcopy(state_dict))
        #else: print(search_name_no_folder)

if len(net_list) < 30: print("No models found.")

In [14]:
def minmax_norm(train_subset, val_subset, test_subset):
    train_tensor, _ = train_subset[:]
    val_tensor, _ = val_subset[:]
    test_tensor, _ = test_subset[:]

    min_val = train_tensor.min(dim=0).values
    max_val = train_tensor.max(dim=0).values
    range_val = (max_val - min_val).clamp(min=1e-6)

    train_norm = ((train_tensor - min_val) / range_val).clamp(0.0, 1.0)
    val_norm   = ((val_tensor   - min_val) / range_val).clamp(0.0, 1.0)
    test_norm  = ((test_tensor  - min_val) / range_val).clamp(0.0, 1.0)

    return train_norm, val_norm, test_norm

In [15]:
results = [[], [], [], [], [], []]
for iter in range(iterations):
    #print(f"Iteration:{iter + 1}/{iterations}")
    seed = iter + 1
    print(f"Seed:{seed} -> ",end='', flush=True)
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    # DATA SPLIT
    train, val, test = data_splitter(df, target_name, split=split, dataset=dataset, data_config=data_config, seed=seed, dtype=dtype)
    _, train_label = train[:]
    _, val_label = val[:]
    _, test_label = test[:]

    if repr_type == "descriptor":
        train_data, val_data, test_data = minmax_norm(train, val, test)
        train = TensorDataset(train_data, train_label)
        val = TensorDataset(val_data,val_label)
        test = TensorDataset(test_data, test_label)
        
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, drop_last=drop_last)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)

    # TESTING
    model = net
    flag = False
    try:
        loaded_result = model.load_state_dict(net_list[iter])
    except RuntimeError as e:
        print(f"{e}")
        model.load_state_dict(net_list[iter], strict=False)
        flag = True
    model.to(device)

    all_preds, all_targets = test_net(model, device, test_loader, train_config)
    calc_metrics(results, all_preds=all_preds, all_targets=all_targets)

Seed:1 -> acc: 0.8178694158075601 auc: 0.8768036886848768
Seed:2 -> acc: 0.8127147766323024 auc: 0.8683847822121328
Seed:3 -> acc: 0.8694158075601375 auc: 0.8653213398035271
Seed:4 -> acc: 0.8470790378006873 auc: 0.909548780487805
Seed:5 -> acc: 0.9518900343642611 auc: 0.958269708503316
Seed:6 -> acc: 0.8264604810996563 auc: 0.9004956396260744
Seed:7 -> acc: 0.8333333333333334 auc: 0.8629618342085521
Seed:8 -> acc: 0.8487972508591065 auc: 0.874497103315095
Seed:9 -> acc: 0.8213058419243986 auc: 0.872042261697434
Seed:10 -> acc: 0.8573883161512027 auc: 0.8783037475345169
Seed:11 -> acc: 0.8539518900343642 auc: 0.8865821678321677
Seed:12 -> acc: 0.8436426116838488 auc: 0.8937898537134285
Seed:13 -> acc: 0.915807560137457 auc: 0.9467642984365723
Seed:14 -> acc: 0.8608247422680413 auc: 0.8919376693766938
Seed:15 -> acc: 0.8316151202749141 auc: 0.8660408093278463
Seed:16 -> acc: 0.8264604810996563 auc: 0.9049848178137652
Seed:17 -> acc: 0.8144329896907216 auc: 0.877274456704192
Seed:18 -> a

In [19]:
with torch.no_grad():
    model.eval()
    for data, targets in test_loader:
        data = data.to(device).unsqueeze(1)

        targets = targets.to(device)
        
        # forward pass
        #test_spk, _ = net(data.view(data.size(0), -1))
        test_spk, _ = model(data)
        print(test_spk.sum(dim=0))

tensor([[4., 1.],
        [0., 1.],
        [1., 0.],
        [3., 1.],
        [1., 0.],
        [3., 3.],
        [1., 1.],
        [2., 3.],
        [0., 1.],
        [2., 2.],
        [1., 2.],
        [0., 3.],
        [3., 2.],
        [0., 0.],
        [3., 3.],
        [1., 3.]], device='cuda:0')
tensor([[2., 0.],
        [3., 1.],
        [2., 3.],
        [3., 2.],
        [3., 1.],
        [0., 0.],
        [0., 0.],
        [2., 3.],
        [1., 2.],
        [3., 1.],
        [0., 3.],
        [3., 1.],
        [1., 1.],
        [2., 1.],
        [2., 2.],
        [0., 1.]], device='cuda:0')
tensor([[3., 3.],
        [4., 0.],
        [3., 1.],
        [2., 1.],
        [4., 1.],
        [3., 1.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [4., 1.],
        [3., 1.],
        [3., 2.],
        [4., 2.],
        [1., 2.],
        [1., 3.],
        [1., 3.]], device='cuda:0')
tensor([[1., 1.],
        [2., 3.],
        [0., 1.],
        [3., 2.],
        [0

#### Save Metrics

In [ ]:
metrics_np = np.zeros(12)

for i, metric in enumerate(results):
    metrics_np[i*2] = np.round(np.mean(metric), 3)
    metrics_np[i*2+1] = np.round(np.std(metric), 3)

# Print Results
if repr_type == "fp":
    if mix:
        print(f"{net_type}_{fp_type}_morgan")
    else:
        print(f"{net_type}_{fp_type}")
else:
    if mix:
        print(f"{net_type}_desc")


print(f"Accuracy:  {metrics_np[0]:.3f} ± {metrics_np[1]:.3f}")
print(f"AUC ROC: {metrics_np[2]:.3f} ± {metrics_np[3]:.3f}")
print(f"Sensitivity: {metrics_np[4]:.3f} ± {metrics_np[5]:.3f}")
print(f"Specificity: {metrics_np[6]:.3f} ± {metrics_np[7]:.3f}")


metric_names = ['Acc', 'AUC', 'Sn', 'Sp', 'F1', 'Precision']
metrics_np = metrics_np.reshape(1, -1)
columns = []
for name in metric_names:
    columns.extend([f'Mean {name}', f'Std {name}'])


df_metrics = pd.DataFrame(metrics_np, columns=columns)
num_hidden = net_config['num_hidden']
time_steps = train_config['num_steps']
num_epochs = train_config['num_epochs']

df_raw = pd.DataFrame({name: results[i] for i, name in enumerate(metric_names)})
df_raw["Seed"] = list(range(1, 31))
df_raw = df_raw[["Seed"] + metric_names]  # reorder columns

blank = pd.DataFrame([[""] * 12] * 3, columns=columns) 


SNN_maccs
Accuracy:  0.915 ± 0.024
AUC ROC: 0.906 ± 0.033
Sensitivity: 0.745 ± 0.097
Specificity: 0.947 ± 0.020


In [ ]:
save = True
filename = make_filename(dirname, target_name, net_type, data_config, lr, weight_decay, optim_type, net_config, train_config, model)
save = not flag

filename = filename.replace("results", "final_results")
if calculate_scores:
    filename = filename.replace(".csv", "_scores.scv")
else:
    filename = filename.replace(".csv", "_no_scores.scv")
if save: 
    df_metrics.to_csv(filename, index=False)
    blank.to_csv(filename, mode='a', index=False, header=False)
    df_raw.to_csv(filename, mode='a', index=False)

print(filename)

results_2\tox21\SR-ARE_SNN_beta-0.95_maccs_167_l11024_t10_e100_b16_lr0.0001_count_loss_Adam_wd0_bias_scores.scv
